First three blocks are installs / imports for libraries

In [44]:
!pip install -U google-generativeai

In [45]:
!pip install PyPDF2 python-docx

In [46]:
import os
import textwrap
import PyPDF2 # For PDF files | Probably don't need
import docx # For .docx Word files | Probably don't need
from IPython.display import display, Markdown, clear_output # For better Colab output
from PIL import Image
from io import BytesIO
from google import genai
from google.colab import userdata
from google.genai import types

Defining functions

In [47]:
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

client = genai.Client(api_key=GOOGLE_API_KEY)

#Names of the models for each part
text_gen_model = "gemini-1.5-flash"
image_gen_model = "gemini-2.0-flash-exp-image-generation"

#Extracts text from a text (.txt) file
def get_txt(txt_path):
    """Extracts text from a plain text file."""
    try:
        with open(txt_path, 'r', encoding='utf-8') as file:
            return file.read()
    except UnicodeDecodeError:
        print(f"Error: Could not decode text file '{txt_path}' with UTF-8. Trying with 'latin-1'.")
        with open(txt_path, 'r', encoding='latin-1') as file:
            return file.read()
    except Exception as e:
        print(f"An error occurred while reading text file '{txt_path}': {e}")
        return None

#Given an image path, loads an image
def get_img(img_path):
  try:
    style_ex = Image.open(img_path)
  except FileNotFoundError:
    print(f"Error: {img_path} could not be found. Please try again")
    style_ex = Image.new('RGB', (100, 100), color = 'red')
    print("Creating replacement image, result will not be styled properly")

  return style_ex

Generative Functions

In [52]:
"""
Text Generation | Programmed so that it can generate a story, and a title for the story.
The function needs to be called separately for generating both.
Parameters:
- out_type : "STORY" by default, generates the story : "TITLE" generates a story based on the 'prompt' input
- prompt : if out_type is "STORY", insert the prompt for the story : if out_type is "TITLE", insert the story
- ex_txt : Example text to be used for the story's format (ONLY ONE) : Value doesn't matter when out_type is "TITLE"
- count : When out_type is "STORY", the story will be exactly 'count' sentences long : When out_type is "TITLE", the title will be exactly 'count' words long
"""
def GenText(prompt, ex_txt, count, out_type="STORY"):
  if out_type.upper() == "STORY":
    final_prompt = (f"""
      Please use the following text as a style and tone guide:
      ---
      {ex_txt}
      ---

      Now, inspired by this style, write a short story in {count} sentences, each sentence on a new line, about: {prompt}
    """)
  elif out_type.upper() == "TITLE":
    final_prompt = (f"""
      Please read the following children story, and create a title for it that is exactly {count} words: {prompt}
    """)

  response = client.models.generate_content(
      model = text_gen_model,
      contents = final_prompt,
      config = types.GenerateContentConfig(
          response_modalities=['TEXT', 'Text']
      )
  )
  return response

"""
Image Generation | Generates an image
Parameters:
- prompt : The text prompt to create an image from
- ex_img : The image fed into the model as a style example (ONLY ONE IMAGE)
"""

def GenImage(prompt, ex_img):
  final_prompt = "In a style inspired by the provided image, create an image for this caption: " + prompt
  feed = (final_prompt, ex_img)

  response = client.models.generate_content(
      model = image_gen_model,
      contents = feed,
      config = types.GenerateContentConfig(
          response_modalities=['TEXT', "IMAGE"]
      ),
  )

  for part in response.candidates[0].content.parts:
    if part.text is not None:
      print(part.text)
    elif part.inline_data is not None:
      image = Image.open(BytesIO((part.inline_data.data)))
      image.save("itjoever.png")
      image.show()

"""
Image Generation With Multiple Examples
Parameters:
- prompt : The text prompt to create an image from
- ex_imgs : A list of images fed into the model as a style example (MULTIPLE ALLOWED | CAN DO ONE)
"""
def GenImage_FromList(prompt, ex_imgs):
  final_prompt = "In a style inspired by the provided images, create an image for this caption: " + prompt
  feed = tuple([final_prompt] + ex_imgs)

  response = client.models.generate_content(
      model = image_gen_model,
      contents = feed,
      config = types.GenerateContentConfig(
          response_modalities=['TEXT', "IMAGE"]
      ),
  )

  for part in response.candidates[0].content.parts:
    if part.text is not None:
      print(part.text)
    elif part.inline_data is not None:
      image = Image.open(BytesIO((part.inline_data.data)))
      image.save("itjoever.png")
      image.show()

In [53]:
#Generates the story
text_sample = get_txt("/content/fishi.txt")
story_prompt = "A frog goes to a waterpark for a day, and meets a cute toad"
story = GenText(story_prompt, text_sample, 20)

for part in story.candidates[0].content.parts:
  if part.text is not None:
    story_str = part.text

story_list = story_str.splitlines()
#Prints the story line by line
for i in range(0, len(story_list)):
  print(str(i) + ": " + story_list[i])

0: Green frog, happy frog, bouncy frog, went to the park.
1: Splash!  He jumped in the big blue pool.
2: Whizz! Down a twisting slide he flew.
3: A little toad, a cute, small toad, sat nearby.
4: Her skin was speckled, a pretty brown.
5: He saw her smile, a tiny, sweet smile.
6: Ribbit! He croaked a hello, a friendly hello.
7: She hopped closer, a quick little hop.
8: They played on the lily pad raft.
9: They zoomed down the water slide, together.
10: Round and round the lazy river they drifted.
11: He splashed water on her, a playful splash.
12: She giggled, a tiny, happy giggle.
13: The sun shone bright, a warm, summer sun.
14: He felt his heart thump, thump, thump.
15: She seemed happy too, her eyes bright.
16: He wished the day would never end.
17: They shared a tasty fly, a juicy, plump fly.
18: It was the best day ever, ever, ever.
19: A new friend made, a summer day's joy.


In [54]:
#Generates a title and prints it
title = GenText(story_str, None, 5, "TITLE")

for part in title.candidates[0].content.parts:
  if part.text is not None:
    final_title = part.text

print(final_title)

Green Frog's Best Summer Day Ever



In [55]:
#Gets an image and generates a new one
img_sample = get_img("/content/famguy.jpg")
GenImage(story_list[0], img_sample)

I will generate an image of a cheerful, bright green frog with wide, happy eyes and a big smile, captured mid-bounce with its legs playfully extended. The frog will be in a vibrant park setting, with lush green grass, colorful flowers, and perhaps a bright blue sky with fluffy white clouds in the background, maintaining the cartoonish style of the provided image.

